In [1]:
import numpy as np
import pandas as pd
import pickle
from tempfile import mkdtemp
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score,make_scorer
from sklearn.preprocessing import StandardScaler,scale as scaler
from sklearn.model_selection import GridSearchCV
import math
from IPython.display import display, HTML

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Convolution1D, MaxPooling1D, LeakyReLU
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.wrappers.scikit_learn import KerasRegressor
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from collections import namedtuple
from keras import losses
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
def split_into_chunks(data, train, predict, step, binary=True, scale=False):
    X, Y = [], []
    for i in range(0, len(data), step):
        try:
            x_i = data[i:i+train]
            y_i = data[i+train+predict]
            
            # Use it only for daily return time series
            if binary:
                if y_i > 0.:
                    y_i = [1., 0.]
                else:
                    y_i = [0., 1.]

                if scale: x_i = scaler(x_i)
                
            else:
                
                timeseries =np.array(data[i:i+train+predict])
                
                if scale: 
                    timeseries = scaler(timeseries)
                x_i = timeseries[:-1]
                y_i = timeseries[-1]
            
        except  Exception as e:
            print (str(e))
            break

        X.append(x_i)
        Y.append(y_i)

    return X, Y

def train_test_split(X, y, train_percentage):
    rows=len(X)
    train_size=int(rows*train_percentage)
    X_train=X[:train_size]
    y_train=y[:train_size]
    X_test=X[train_size:]
    y_test=y[train_size:]
    return X_train,y_train,X_test,y_test


In [3]:
Dataset=namedtuple('Dataset','exchange df')
DatasetMLModel= namedtuple('DatasetMLModel','exchange  train_size tscv_split test_size X_train y_train X_test y_test scaler_features scaler_target')
Regressor= namedtuple('Regressor','name regressor_class params type')
FeatureSelection= namedtuple('FeatureSelection','dataset regressor params RFECV')
TRAIN_PERCENTAGE=0.8
TRAIN_SIZE_FEATURES = 20
EVALUATION_PERCENT=0.2
TARGET_TIME = 1
LAG_SIZE = 1
EMB_SIZE = 1


with open('datasets/log_divided_close_datasets.pkl', 'rb') as input4:
    log_divided_close_datasets = pickle.load(input4)


    
dataset=log_divided_close_datasets['btc_brl']
data=dataset.df['log_return']

rows,=data.shape
train_size=int(rows*TRAIN_PERCENTAGE)
scaler = StandardScaler()
scaler.fit(data[:train_size+TRAIN_SIZE_FEATURES].values.reshape(-1, 1))
data=scaler.transform(data.values.reshape(-1, 1))
data=data.reshape(1,-1).tolist()[0]

In [4]:
data[20]

-4.281966288117148

In [5]:

X,Y =split_into_chunks(data, TRAIN_SIZE_FEATURES,TARGET_TIME, LAG_SIZE, binary=False, scale=False)
X, Y = np.array(X), np.array(Y)
X_train,y_train,X_test,y_test=train_test_split(X, Y, TRAIN_PERCENTAGE)

list index out of range


In [6]:
y_train


array([-4.28196629,  6.70878824, -5.11171855, ...,  0.0928846 ,
       -0.09352972,  0.06764178])

In [7]:
#tscv_split=int(rows//(train_size*EVALUATION_PERCENT)-1)

tscv_split=3

In [8]:
def create_model_deep_learning():  
    model = Sequential()
    model.add(Dense(64, input_shape=(TRAIN_SIZE_FEATURES,),activity_regularizer=regularizers.l2(0.01))) 
    model.add(BatchNormalization()) 
    model.add(LeakyReLU()) 
    model.add(Dense(16,activity_regularizer=regularizers.l2(0.01))) 
    model.add(BatchNormalization()) 
    model.add(LeakyReLU()) 
    model.add(Dense(1)) 
    model.add(Activation('linear'))
    opt = Nadam(lr=0.001)
    model.compile(optimizer='adam', 
              loss='mse')
    return model

def create_model_deep_learning2(size1=64,size2=64,size3=64,activation='relu', lr=0.01,loss=losses.mse):  
    model = Sequential()
    model.add(Dense(size1, input_shape=(TRAIN_SIZE_FEATURES,),activation=activation)) 
    model.add(Dense(size2,activation=activation) )
    model.add(Dense(size3,activation=activation) )
    model.add(Dense(1)) 
    model.add(Activation('linear'))
    opt = Adam(lr=lr)
    model.compile(optimizer=opt, 
              loss=loss)
    return model

In [9]:
def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    score = r2_score(y_true,y_predict)
    return score

In [10]:
model = KerasRegressor(build_fn=create_model_deep_learning2)

In [11]:


batch_size = [10,50]
epochs = [10,20,30]
lr=[0.01,0.1]
size1=[64,128]
size2=[64,128]
size3=[64,128]
activation=['relu', 'sigmoid','tanh','linear']
loss=[losses.logcosh,losses.mse]

param_grid = dict(keras_model=[KerasRegressor(build_fn=create_model_deep_learning2)],
                  keras_model__batch_size=batch_size,
                  keras_model__epochs=epochs,
                  keras_model__lr=lr,
                  keras_model__size1=size1,
                  keras_model__size2=size2,
                  keras_model__size3=size3,
                  keras_model__activation=activation,
                  keras_model__loss=loss
                 )

param_grid

{'keras_model': [<keras.wrappers.scikit_learn.KerasRegressor at 0x10c8c9dd8>],
 'keras_model__activation': ['relu', 'sigmoid', 'tanh', 'linear'],
 'keras_model__batch_size': [10],
 'keras_model__epochs': [20],
 'keras_model__loss': [<function keras.losses.logcosh>,
  <function keras.losses.mean_squared_error>],
 'keras_model__lr': [0.001],
 'keras_model__size1': [64],
 'keras_model__size2': [64],
 'keras_model__size3': [64]}

In [12]:
cachedir = mkdtemp()
estimators= [('keras_model',KerasRegressor(build_fn=create_model_deep_learning2))]
params=[param_grid,]
pipe = Pipeline(estimators, memory=cachedir)
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'keras_model', 'keras_model__build_fn'])

In [13]:
scoring_fnc = make_scorer(performance_metric)
cvts=TimeSeriesSplit(n_splits=tscv_split)
grid = GridSearchCV(estimator=pipe, 
                    param_grid=params,
                    scoring=scoring_fnc,
                    cv=cvts,
                    error_score=np.NaN,
                    verbose=6,
                    n_jobs=1
                   )  
grid = grid.fit(X_train,y_train )

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] keras_model=<keras.wrappers.scikit_learn.KerasRegressor object at 0x10c8c9dd8>, keras_model__activation=relu, keras_model__batch_size=10, keras_model__epochs=20, keras_model__loss=<function logcosh at 0x10c799048>, keras_model__lr=0.001, keras_model__size1=64, keras_model__size2=64, keras_model__size3=64 
Epoch 1/20
334/334 [==============================] - 2s 7ms/step - loss: 0.6115
Epoch 2/20
334/334 [==============================] - 0s 246us/step - loss: 0.5212
Epoch 3/20
334/334 [==============================] - 0s 243us/step - loss: 0.4811
Epoch 4/20
334/334 [==============================] - 0s 244us/step - loss: 0.4461
Epoch 5/20
334/334 [==============================] - 0s 231us/step - loss: 0.4040
Epoch 6/20
334/334 [==============================] - 0s 251us/step - loss: 0.3734
Epoch 7/20
334/334 [==============================] - 0s 235us/step - loss: 0.3369
Epoch 8/20
334/334 [=============================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.7s remaining:    0.0s


Epoch 1/20
665/665 [==============================] - 1s 780us/step - loss: 0.4209
Epoch 2/20
665/665 [==============================] - 0s 211us/step - loss: 0.3811
Epoch 3/20
665/665 [==============================] - 0s 208us/step - loss: 0.3516
Epoch 4/20
665/665 [==============================] - 0s 212us/step - loss: 0.3306
Epoch 5/20
665/665 [==============================] - 0s 204us/step - loss: 0.3028
Epoch 6/20
665/665 [==============================] - 0s 213us/step - loss: 0.2757
Epoch 7/20
665/665 [==============================] - 0s 212us/step - loss: 0.2489
Epoch 8/20
665/665 [==============================] - 0s 214us/step - loss: 0.2304
Epoch 9/20
665/665 [==============================] - 0s 194us/step - loss: 0.2093
Epoch 10/20
665/665 [==============================] - 0s 210us/step - loss: 0.1996
Epoch 11/20
665/665 [==============================] - 0s 210us/step - loss: 0.1821
Epoch 12/20
665/665 [==============================] - 0s 203us/step - loss: 0.1597
E

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    9.7s remaining:    0.0s


Epoch 1/20
996/996 [==============================] - 1s 558us/step - loss: 0.3286
Epoch 2/20
996/996 [==============================] - 0s 205us/step - loss: 0.3044
Epoch 3/20
996/996 [==============================] - 0s 203us/step - loss: 0.2860
Epoch 4/20
996/996 [==============================] - 0s 205us/step - loss: 0.2680
Epoch 5/20
996/996 [==============================] - 0s 214us/step - loss: 0.2518
Epoch 6/20
996/996 [==============================] - 0s 222us/step - loss: 0.2309
Epoch 7/20
996/996 [==============================] - 0s 183us/step - loss: 0.2196
Epoch 8/20
996/996 [==============================] - 0s 225us/step - loss: 0.1988
Epoch 9/20
996/996 [==============================] - 0s 211us/step - loss: 0.1796
Epoch 10/20
996/996 [==============================] - 0s 212us/step - loss: 0.1655
Epoch 11/20
996/996 [==============================] - 0s 207us/step - loss: 0.1584
Epoch 12/20
996/996 [==============================] - 0s 213us/step - loss: 0.1420
E

...

334/334 [==============================] - 0s 267us/step - loss: 0.7761
Epoch 15/20
334/334 [==============================] - 0s 312us/step - loss: 0.6811
Epoch 16/20
334/334 [==============================] - 0s 266us/step - loss: 0.6448
Epoch 17/20
334/334 [==============================] - 0s 289us/step - loss: 0.5640
Epoch 18/20
334/334 [==============================] - 0s 275us/step - loss: 0.4782
Epoch 19/20
334/334 [==============================] - 0s 273us/step - loss: 0.4456
Epoch 20/20
334/334 [==============================] - 0s 313us/step - loss: 0.4091
[CV]  keras_model=<keras.wrappers.scikit_learn.KerasRegressor object at 0x10c8c9dd8>, keras_model__activation=tanh, keras_model__batch_size=10, keras_model__epochs=20, keras_model__loss=<function mean_squared_error at 0x10c79cbf8>, keras_model__lr=0.001, keras_model__size1=64, keras_model__size2=64, keras_model__size3=64, score=-0.5169799760845224, total=   3.4s
[CV] keras_model=<keras.wrappers.scikit_learn.KerasRegresso

Epoch 1/20
665/665 [==============================] - 1s 2ms/step - loss: 0.4896
Epoch 2/20
665/665 [==============================] - 0s 298us/step - loss: 0.4491
Epoch 3/20
665/665 [==============================] - 0s 291us/step - loss: 0.4471
Epoch 4/20
665/665 [==============================] - 0s 301us/step - loss: 0.4263
Epoch 5/20
665/665 [==============================] - 0s 311us/step - loss: 0.4307
Epoch 6/20
665/665 [==============================] - 0s 298us/step - loss: 0.4380
Epoch 7/20
665/665 [==============================] - 0s 304us/step - loss: 0.4255
Epoch 8/20
665/665 [==============================] - 0s 294us/step - loss: 0.4283
Epoch 9/20
665/665 [==============================] - 0s 325us/step - loss: 0.4198
Epoch 10/20
665/665 [==============================] - 0s 300us/step - loss: 0.4219
Epoch 11/20
665/665 [==============================] - 0s 299us/step - loss: 0.4255
Epoch 12/20
665/665 [==============================] - 0s 294us/step - loss: 0.4164
Epo

665/665 [==============================] - 0s 324us/step - loss: 1.6050
Epoch 15/20
665/665 [==============================] - 0s 317us/step - loss: 1.6167
Epoch 16/20
665/665 [==============================] - 0s 284us/step - loss: 1.6336
Epoch 17/20
665/665 [==============================] - 0s 302us/step - loss: 1.5790
Epoch 18/20
665/665 [==============================] - 0s 300us/step - loss: 1.5615
Epoch 19/20
665/665 [==============================] - 0s 308us/step - loss: 1.5645
Epoch 20/20
665/665 [==============================] - 0s 288us/step - loss: 1.5566
[CV]  keras_model=<keras.wrappers.scikit_learn.KerasRegressor object at 0x10c8c9dd8>, keras_model__activation=linear, keras_model__batch_size=10, keras_model__epochs=20, keras_model__loss=<function mean_squared_error at 0x10c79cbf8>, keras_model__lr=0.001, keras_model__size1=64, keras_model__size2=64, keras_model__size3=64, score=0.012318621247757289, total=   6.3s
[CV] keras_model=<keras.wrappers.scikit_learn.KerasRegre

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  2.0min finished


Epoch 1/20
1327/1327 [==============================] - 2s 1ms/step - loss: 0.3453
Epoch 2/20
1327/1327 [==============================] - 0s 292us/step - loss: 0.3119
Epoch 3/20
1327/1327 [==============================] - 0s 288us/step - loss: 0.3024
Epoch 4/20
1327/1327 [==============================] - 0s 292us/step - loss: 0.2942
Epoch 5/20
1327/1327 [==============================] - 0s 287us/step - loss: 0.2929
Epoch 6/20
1327/1327 [==============================] - 0s 286us/step - loss: 0.2877
Epoch 7/20
1327/1327 [==============================] - 0s 287us/step - loss: 0.2914
Epoch 8/20
1327/1327 [==============================] - 0s 290us/step - loss: 0.2854
Epoch 9/20
1327/1327 [==============================] - 0s 292us/step - loss: 0.2836
Epoch 10/20
1327/1327 [==============================] - 0s 290us/step - loss: 0.2840
Epoch 11/20
1327/1327 [==============================] - 0s 276us/step - loss: 0.2807
Epoch 12/20
1327/1327 [==============================] - 0s 258us

In [14]:
columns=['Regressor','Score','BestEstimator']


In [15]:
data_list=[]
data_row=['MLP',
                grid.best_score_,
                grid.best_estimator_
                ]
data_list.append(data_row)
score=pd.DataFrame(data_list,columns=columns)
score.set_index(['Regressor'],inplace=True)

In [16]:
score

,Score,BestEstimator
Regressor,,
MLP,-0.023744,Pipeline(memory='/var/folders/ch/53dqzxsx15b3c...


In [17]:
grid.best_params_

{'keras_model': <keras.wrappers.scikit_learn.KerasRegressor at 0x10c8c9dd8>,
 'keras_model__activation': 'linear',
 'keras_model__batch_size': 10,
 'keras_model__epochs': 20,
 'keras_model__loss': <function keras.losses.logcosh>,
 'keras_model__lr': 0.001,
 'keras_model__size1': 64,
 'keras_model__size2': 64,
 'keras_model__size3': 64}

In [31]:
batch_size = [10]
epochs = [20]
lr=[0.001]
size1=[64]
size2=[64]
size3=[64]
activation=['linear']
loss=[losses.logcosh]
pipe = Pipeline(estimators)
param_grid = dict(
                 batch_size=batch_size,
                 epochs=epochs,
                 lr=lr,
                 size1=size1,
                 size2=size2,
                 size3=size3,
                 activation=activation,
                 loss=loss
                 )


grid2 = GridSearchCV(estimator=KerasRegressor(build_fn=create_model_deep_learning2), 
                    param_grid=param_grid,
                    scoring=scoring_fnc,
                    cv=cvts,
                    error_score=np.NaN,
                    verbose=6,
                    n_jobs=1
                   )  

grid2 = grid2.fit(X_train,y_train )

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] activation=linear, batch_size=10, epochs=20, loss=<function logcosh at 0x10c799048>, lr=0.001, size1=64, size2=64, size3=64 
Epoch 1/20
334/334 [==============================] - 3s 8ms/step - loss: 0.7053
Epoch 2/20
334/334 [==============================] - 0s 417us/step - loss: 0.6438
Epoch 3/20
334/334 [==============================] - 0s 433us/step - loss: 0.6151
Epoch 4/20
334/334 [==============================] - 0s 429us/step - loss: 0.6310
Epoch 5/20
334/334 [==============================] - 0s 454us/step - loss: 0.6180
Epoch 6/20
334/334 [==============================] - 0s 427us/step - loss: 0.5911
Epoch 7/20
334/334 [==============================] - 0s 449us/step - loss: 0.6005
Epoch 8/20
334/334 [==============================] - 0s 433us/step - loss: 0.5665
Epoch 9/20
334/334 [==============================] - 0s 438us/step - loss: 0.5967
Epoch 10/20
334/334 [==============================] - 0s 443us/st

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.4s remaining:    0.0s


Epoch 1/20
665/665 [==============================] - 3s 5ms/step - loss: 0.5348
Epoch 2/20
665/665 [==============================] - 0s 378us/step - loss: 0.4570
Epoch 3/20
665/665 [==============================] - 0s 373us/step - loss: 0.4578
Epoch 4/20
665/665 [==============================] - 0s 376us/step - loss: 0.4347
Epoch 5/20
665/665 [==============================] - 0s 380us/step - loss: 0.4316
Epoch 6/20
665/665 [==============================] - 0s 384us/step - loss: 0.4285
Epoch 7/20
665/665 [==============================] - 0s 377us/step - loss: 0.4252
Epoch 8/20
665/665 [==============================] - 0s 449us/step - loss: 0.4224
Epoch 9/20
665/665 [==============================] - 0s 474us/step - loss: 0.4272
Epoch 10/20
665/665 [==============================] - 0s 472us/step - loss: 0.4221
Epoch 11/20
665/665 [==============================] - 0s 480us/step - loss: 0.4182
Epoch 12/20
665/665 [==============================] - 0s 425us/step - loss: 0.4268
Epo

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   17.7s remaining:    0.0s


Epoch 1/20
996/996 [==============================] - 3s 3ms/step - loss: 0.3937
Epoch 2/20
996/996 [==============================] - 0s 379us/step - loss: 0.3537
Epoch 3/20
996/996 [==============================] - 0s 369us/step - loss: 0.3445
Epoch 4/20
996/996 [==============================] - 0s 374us/step - loss: 0.3375
Epoch 5/20
996/996 [==============================] - 0s 368us/step - loss: 0.3345
Epoch 6/20
996/996 [==============================] - 0s 361us/step - loss: 0.3317
Epoch 7/20
996/996 [==============================] - 0s 364us/step - loss: 0.3323
Epoch 8/20
996/996 [==============================] - 0s 376us/step - loss: 0.3263
Epoch 9/20
996/996 [==============================] - 0s 441us/step - loss: 0.3350
Epoch 10/20
996/996 [==============================] - 0s 411us/step - loss: 0.3268
Epoch 11/20
996/996 [==============================] - 0s 465us/step - loss: 0.3262
Epoch 12/20
996/996 [==============================] - 0s 392us/step - loss: 0.3261
Epo

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   30.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   30.3s finished


Epoch 1/20
1327/1327 [==============================] - 3s 2ms/step - loss: 0.3264
Epoch 2/20
1327/1327 [==============================] - 1s 382us/step - loss: 0.3086
Epoch 3/20
1327/1327 [==============================] - 1s 380us/step - loss: 0.3022
Epoch 4/20
1327/1327 [==============================] - 1s 385us/step - loss: 0.2902
Epoch 5/20
1327/1327 [==============================] - 1s 386us/step - loss: 0.2925
Epoch 6/20
1327/1327 [==============================] - 1s 393us/step - loss: 0.2879
Epoch 7/20
1327/1327 [==============================] - 1s 395us/step - loss: 0.2853
Epoch 8/20
1327/1327 [==============================] - 1s 383us/step - loss: 0.2837
Epoch 9/20
1327/1327 [==============================] - 1s 398us/step - loss: 0.2819
Epoch 10/20
1327/1327 [==============================] - 1s 389us/step - loss: 0.2844
Epoch 11/20
1327/1327 [==============================] - 1s 389us/step - loss: 0.2777
Epoch 12/20
1327/1327 [==============================] - 1s 387us

In [20]:
data_list=[]
data_row=['MLP',
                grid2.best_score_,
                grid2.best_estimator_
                ]
data_list.append(data_row)
score=pd.DataFrame(data_list,columns=columns)
score.set_index(['Regressor'],inplace=True)


In [32]:
grid2.best_params_

{'activation': 'linear',
 'batch_size': 10,
 'epochs': 20,
 'loss': <function keras.losses.logcosh>,
 'lr': 0.001,
 'size1': 64,
 'size2': 64,
 'size3': 64}

In [36]:
#https://stackoverflow.com/questions/40396042/how-to-save-scikit-learn-keras-model-into-a-persistence-file-pickle-hd5-json-ya
def save_model(model):
    # saving model
    json_model = model.to_json()
    open('models/deep_learning_arch.json', 'w').write(json_model)
    # saving weights
    model.save_weights('models/deep_learning_weights.h5', overwrite=True)

def load_model():
    # loading model
    model = model_from_json(open('models/deep_learning_arch.json').read())
    model.load_weights('models/model_weights.h5')
    model.compile(loss=losses.logcosh, optimizer=Adam(lr=0.001))#Dependencies
    return model
    
save_model(grid2.best_estimator_.model)